In [1]:
import os
import time

import numpy as np
import torch
import argparse

from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer, 
                          Phi3ForCausalLM,
                          Phi3Config,
                          pipeline,
                          )
from transformers.pipelines.pt_utils import KeyDataset
from huggingface_hub import hf_hub_download
from vllm import LLM, SamplingParams

import utils

In [ ]:
def run_gguf_inference(model_path):
    PROMPT_TEMPLATE = "<|system|>\n{system_message}</s>\n<|user|>\n{prompt}</s>\n<|assistant|>\n"  # noqa: E501
    system_message = "You are a friendly chatbot who always responds in the style of a pirate."  # noqa: E501
    # Sample prompts.
    prompts = [
        "How many helicopters can a human eat in one sitting?",
        "What's the future of AI?",
    ]
    prompts = [
        PROMPT_TEMPLATE.format(system_message=system_message, prompt=prompt)
        for prompt in prompts
    ]
    # Create a sampling params object.
    sampling_params = SamplingParams(temperature=0, max_tokens=128)

    # Create an LLM.
    llm = LLM(model=model_path,
              tokenizer="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
              gpu_memory_utilization=0.95)

    outputs = llm.generate(prompts, sampling_params)
    # Print the outputs.
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")


if __name__ == "__main__":
    repo_id = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"
    filename = "tinyllama-1.1b-chat-v1.0.Q4_0.gguf"
    model = hf_hub_download(repo_id, filename=filename)
    run_gguf_inference(model)

In [ ]:

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="vLLM + MInference")
    parser.add_argument("--model_path", type=str, default="models/")
    parser.add_argument("--model_name", type=str, default="Phi-3-medium-4k-instruct")
    parser.add_argument("--data_path", type=str, default="data/")
    parser.add_argument("--dataset", type=str, default="test_dataset.jsonl")
    parser.add_argument('--seed',type=int, default=0)
    parser.add_argument("--batch_size", type=int, default=1)
    parser.add_argument('--dtype',type=str, default="auto")
    parser.add_argument('--gpu_memory_utilization',type=float, default=0.9) # 0.3, 0.5
    parser.add_argument('--enforce_eager',type=bool, default=False)
    
    parser.add_argument('--n', type=int, default=1)
    parser.add_argument('--temperature', type=float, default=0.0)
parser.add_argument('--top_p', type=float, default=1.0)
parser.add_argument('--top_k', type=float, default=-1)
parser.add_argument('--ignore_eos', type=bool, default=True)
parser.add_argument('--max_tokens', type=int, default=500)

config = parser.parse_args([])
utils.seed_everything(config.seed)
CURR_PATH = os.getcwd()